# 🚀 Grafo V3: Evaluación Booleana Exhaustiva

Este notebook implementa **la versión más robusta** con evaluación explícita de TODOS los códigos.

## 🎯 Mejoras sobre V2

**Problema en V2:** Si encuentra 1 código histórico, NO genera nuevos → Pierde conceptos

**Solución en V3:**
- ✅ Evalúa **TODOS** los códigos con True/False + confianza
- ✅ Identifica **gaps de cobertura** (qué NO está cubierto)
- ✅ Captura casos **mixtos** naturalmente

---


## 🔧 Instalación de Dependencias

Si encuentras errores de import, ejecuta esta celda:


In [15]:
# Ejecuta esto si tienes errores de import
%pip install grandalf langchain langchain-openai langgraph python-dotenv pandas openpyxl tenacity

print("💡 Si necesitas instalar dependencias, descomenta la línea de arriba")


Note: you may need to restart the kernel to use updated packages.
💡 Si necesitas instalar dependencias, descomenta la línea de arriba



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# 🛠️ UTILIDADES Y MEJORAS

from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import re
from typing import Tuple, Optional, List, Dict

# ========== 1. RETRY LOGIC ==========
def llamada_llm_con_retry(chain, input_data, max_intentos=3):
    """
    Ejecuta una llamada al LLM con retry automático en caso de error.
    
    Args:
        chain: Cadena de LangChain a ejecutar
        input_data: Datos de entrada para el chain
        max_intentos: Número máximo de intentos (default: 3)
    
    Returns:
        Resultado de la llamada al LLM
    """
    @retry(
        stop=stop_after_attempt(max_intentos),
        wait=wait_exponential(multiplier=1, min=2, max=10),
        retry=retry_if_exception_type((Exception,)),
        reraise=True
    )
    def _intentar_llamada():
        try:
            return chain.invoke(input_data)
        except Exception as e:
            print(f"   ⚠️  Error en LLM: {type(e).__name__}, reintentando...")
            raise
    
    return _intentar_llamada()

# ========== 2. DETECCIÓN TEMPRANA DE RESPUESTAS ESPECIALES ==========
PATRONES_ESPECIALES = {
    # Patrones para código 90: Ninguno
    r'\b(ninguno|ninguna|nada de eso|ninguno de|ninguna de)\b': 90,
    # Patrones para código 91: No Recuerda
    r'\b(no recuerdo|no me acuerdo|no recuerdo|olvidé|olvidé)\b': 91,
    # Patrones para código 92: No Sabe
    r'\b(no sé|no se|no conozco|no tengo idea|no lo sé|no lo se)\b': 92,
    # Patrones para código 93: No Responde
    r'^[\s\-\.]+$|^$': 93,  # Vacío, solo espacios, guiones o puntos
    # Patrones para código 94: Cualquiera
    r'\b(cualquiera|cualquier|da igual|me da igual|es igual)\b': 94,
    # Patrones para código 95: Todos
    r'\b(todos|todas|todos los|todas las)\b': 95,
    # Patrones para código 96: No Aplica
    r'\b(no aplica|no corresponde|no es para mí|no es mi caso)\b': 96,
    # Patrones para código 97: Ningún Otro
    r'\b(ningún otro|ninguna otra|ninguno más|ninguna más)\b': 97,
    # Patrones para código 98: Nada
    r'^\s*(nada|nada más|nada en particular|nada especial)\s*$': 98,
}

def detectar_codigo_especial(texto: str) -> Optional[int]:
    """
    Detecta si una respuesta corresponde a un código especial (90-98).
    
    Args:
        texto: Texto de la respuesta a analizar
    
    Returns:
        Código especial si se detecta, None en caso contrario
    """
    texto_lower = texto.lower().strip()
    
    # Verificar patrones
    for patron, codigo in PATRONES_ESPECIALES.items():
        if re.search(patron, texto_lower, re.IGNORECASE):
            return codigo
    
    return None

# ========== 3. DEDUPLICACIÓN DE CÓDIGOS SIMILARES ==========
def normalizar_texto(texto: str) -> str:
    """Normaliza texto para comparación (lowercase, sin acentos básicos)"""
    texto = texto.lower().strip()
    # Reemplazos básicos
    texto = texto.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
    return texto

def son_conceptos_similares(desc1: str, desc2: str) -> bool:
    """
    Determina si dos descripciones representan el mismo concepto.
    
    Args:
        desc1: Primera descripción
        desc2: Segunda descripción
    
    Returns:
        True si son similares, False en caso contrario
    """
    # Definir palabras comunes al inicio
    palabras_comunes = ["para", "de", "en", "con", "sin", "por", "la", "el", "un", "una"]
    
    desc1_norm = normalizar_texto(desc1)
    desc2_norm = normalizar_texto(desc2)
    
    # Coincidencia exacta
    if desc1_norm == desc2_norm:
        return True
    
    # Una contiene a la otra (concepto más general vs específico)
    if desc1_norm in desc2_norm or desc2_norm in desc1_norm:
        # Verificar que no sea solo una palabra común
        if desc1_norm not in palabras_comunes and desc2_norm not in palabras_comunes:
            return True
    
    # Palabras clave similares (ej: "saludable", "salud", "salubre")
    palabras1 = set(desc1_norm.split())
    palabras2 = set(desc2_norm.split())
    
    # Si comparten más del 50% de palabras significativas
    palabras_significativas = palabras1 | palabras2
    palabras_significativas = {p for p in palabras_significativas if len(p) > 3 and p not in palabras_comunes}
    
    if palabras_significativas:
        interseccion = palabras1 & palabras2
        if len(interseccion) / len(palabras_significativas) > 0.5:
            return True
    
    return False

def deduplicar_codigos_batch(conceptos_batch: List[Dict]) -> List[Dict]:
    """
    Deduplica códigos similares dentro de un batch, agrupándolos bajo el mismo código.
    
    Args:
        conceptos_batch: Lista de conceptos nuevos generados en el batch
    
    Returns:
        Lista de conceptos deduplicados
    """
    if not conceptos_batch:
        return []
    
    # Agrupar conceptos similares
    grupos = []
    codigos_usados = {}  # descripcion_normalizada -> (codigo, descripcion_final)
    
    for concepto in conceptos_batch:
        desc = concepto.get("descripcion", "")
        codigo = concepto.get("codigo")
        
        # Buscar si ya existe un concepto similar
        encontrado = False
        for desc_existente, (cod_existente, desc_final) in codigos_usados.items():
            if son_conceptos_similares(desc, desc_existente):
                # Usar el código existente
                concepto["codigo"] = cod_existente
                concepto["descripcion"] = desc_final  # Usar la descripción más general
                encontrado = True
                break
        
        if not encontrado:
            # Nuevo concepto único
            desc_norm = normalizar_texto(desc)
            codigos_usados[desc_norm] = (codigo, desc)
    
    return conceptos_batch

# ========== 4. VALIDACIÓN Y CORRECCIÓN DE CÓDIGOS GENERADOS ==========
def validar_y_corregir_codigos(conceptos_nuevos: List[Dict], codigo_base: int) -> Tuple[List[Dict], int]:
    """
    Valida y corrige códigos generados para asegurar secuencialidad.
    
    Args:
        conceptos_nuevos: Lista de conceptos nuevos generados por el LLM
        codigo_base: Código base esperado para empezar
    
    Returns:
        Tupla con (conceptos_corregidos, siguiente_codigo)
    """
    if not conceptos_nuevos:
        return [], codigo_base
    
    conceptos_corregidos = []
    codigo_actual = codigo_base
    
    # Ordenar por código original para mantener orden lógico
    conceptos_ordenados = sorted(conceptos_nuevos, key=lambda x: x.get("codigo", codigo_actual))
    
    for concepto in conceptos_ordenados:
        codigo_original = concepto.get("codigo", codigo_actual)
        
        # Si el código está fuera de secuencia, corregirlo
        if codigo_original < codigo_base:
            print(f"   ⚠️  Código {codigo_original} fuera de secuencia, corrigiendo a {codigo_actual}")
            concepto["codigo"] = codigo_actual
        elif codigo_original != codigo_actual:
            # Si hay un salto, usar el código actual
            print(f"   ⚠️  Salto en secuencia: {codigo_original} -> {codigo_actual}, corrigiendo")
            concepto["codigo"] = codigo_actual
        
        conceptos_corregidos.append(concepto)
        codigo_actual += 1
    
    return conceptos_corregidos, codigo_actual

# ========== 4. MÉTRICAS Y LOGGING ==========
def inicializar_metricas() -> Dict:
    """Inicializa el diccionario de métricas"""
    return {
        "inicio_tiempo": time.time(),
        "llamadas_llm": 0,
        "errores_llm": 0,
        "reintentos": 0,
        "respuestas_especiales_detectadas": 0,
        "codigos_corregidos": 0,
        "tokens_estimados": 0,
        "tiempo_por_nodo": {}
    }

def registrar_tiempo_nodo(metricas: Dict, nombre_nodo: str, tiempo: float):
    """Registra el tiempo de ejecución de un nodo"""
    if nombre_nodo not in metricas["tiempo_por_nodo"]:
        metricas["tiempo_por_nodo"][nombre_nodo] = []
    metricas["tiempo_por_nodo"][nombre_nodo].append(tiempo)

def imprimir_metricas(metricas: Dict):
    """Imprime un resumen de las métricas"""
    tiempo_total = time.time() - metricas["inicio_tiempo"]
    print(f"\n{'='*60}")
    print(f"📊 MÉTRICAS DE EJECUCIÓN")
    print(f"{'='*60}")
    print(f"⏱️  Tiempo total: {tiempo_total:.2f}s")
    print(f"🤖 Llamadas LLM: {metricas['llamadas_llm']}")
    print(f"❌ Errores LLM: {metricas['errores_llm']}")
    print(f"🔄 Reintentos: {metricas['reintentos']}")
    print(f"🎯 Respuestas especiales detectadas: {metricas['respuestas_especiales_detectadas']}")
    print(f"🔧 Códigos corregidos: {metricas['codigos_corregidos']}")
    
    if metricas["tiempo_por_nodo"]:
        print(f"\n⏱️  Tiempo por nodo:")
        for nodo, tiempos in metricas["tiempo_por_nodo"].items():
            promedio = sum(tiempos) / len(tiempos)
            total = sum(tiempos)
            print(f"   • {nodo}: {promedio:.2f}s promedio ({total:.2f}s total, {len(tiempos)} ejecuciones)")

print("✅ Utilidades y mejoras cargadas")


✅ Utilidades y mejoras cargadas


In [17]:
import os
import sys
from pathlib import Path
from datetime import datetime
from typing import TypedDict, List, Dict, Literal, Optional
import pandas as pd
import time

# Configurar paths
project_root = Path.cwd().parent
sys.path.append(str(project_root / "backend" / "src"))

# Cargar variables de entorno
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

# Verificar API key
assert os.getenv("OPENAI_API_KEY"), "❌ Falta OPENAI_API_KEY en .env"

# Imports de LangChain
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

print("✅ Setup completo")
print(f"📂 Ruta del proyecto: {project_root}")


✅ Setup completo
📂 Ruta del proyecto: c:\Users\ivan\Documents\cod-script


In [18]:
# CONFIGURACIÓN
ARCHIVO_RESPUESTAS = project_root / "temp" / "P2 - copia.xlsx"
USAR_CATALOGO_HISTORICO = False
ARCHIVO_CATALOGO = project_root / "result" / "modelos" / "catalogo_propuestos.xlsx"
MAX_RESPUESTAS = None
BATCH_SIZE = 10

# 🆕 MODO DATO AUXILIAR (Dinámico - sin categorías hardcodeadas)
# Formato del archivo con dato auxiliar:
# - Columna 1: ID
# - Columna 2: Pregunta (opcional, para referencia)
# - Columna 3: Respuesta abierta
# - Columna 4: Dato auxiliar (categoría - puede ser cualquier valor: "Desconfianza", "Felicidad", "Categoría A", etc.)
# 
# El sistema identificará automáticamente las categorías desde el dato auxiliar y generará
# códigos secuenciales independientes para cada categoría.
USAR_DATO_AUXILIAR = False  # Cambiar a True si el archivo tiene datos auxiliares

# Modelos disponibles: "gpt-5", "gpt-4o", "gpt-4o-mini", "gpt-4-turbo"
MODELO_GPT = "gpt-5"  # 🆕 GPT-5 disponible desde 2025

# CÓDIGOS ESPECIALES (siempre disponibles)
CODIGOS_ESPECIALES = [
    {"codigo": 90, "descripcion": "Ninguno", "seccion": None, "seccion_clave": None},
    {"codigo": 91, "descripcion": "No Recuerda", "seccion": None, "seccion_clave": None},
    {"codigo": 92, "descripcion": "No Sabe", "seccion": None, "seccion_clave": None},
    {"codigo": 93, "descripcion": "No Responde", "seccion": None, "seccion_clave": None},
    {"codigo": 94, "descripcion": "Cualquiera", "seccion": None, "seccion_clave": None},
    {"codigo": 95, "descripcion": "Todos", "seccion": None, "seccion_clave": None},
    {"codigo": 96, "descripcion": "No Aplica", "seccion": None, "seccion_clave": None},
    {"codigo": 97, "descripcion": "Ningún Otro", "seccion": None, "seccion_clave": None},
    {"codigo": 98, "descripcion": "Nada", "seccion": None, "seccion_clave": None}
]

print(f"📄 Archivo: {ARCHIVO_RESPUESTAS.name}")
print(f"📚 Catálogo: {'✅ Sí' if USAR_CATALOGO_HISTORICO else '❌ No'}")
print(f"🤖 Modelo: {MODELO_GPT}")
print(f"🔢 Códigos especiales: {len(CODIGOS_ESPECIALES)} (90-98)")

📄 Archivo: P2 - copia.xlsx
📚 Catálogo: ❌ No
🤖 Modelo: gpt-5
🔢 Códigos especiales: 9 (90-98)


In [19]:
def cargar_respuestas(archivo_excel, max_respuestas=None):
    """
    Carga respuestas del Excel
    
    Formato:
    - Primera columna = ID
    - Segunda columna = Respuestas
    """
    df = pd.read_excel(archivo_excel)
    
    # Primera columna = ID
    columna_id = df.columns[0]
    # Segunda columna = Respuestas
    columna_respuestas = df.columns[1]
    nombre_pregunta = columna_respuestas
    
    respuestas = []
    for idx, row in df.iterrows():
        texto = str(row[columna_respuestas]).strip()
        if texto and texto.lower() not in ['nan', 'none', '-', '.']:
            # Obtener ID de la primera columna
            id_valor = row[columna_id]
            if pd.isna(id_valor):
                id_valor = idx + 1
            
            respuestas.append({
                "id": id_valor,
                "fila_excel": idx + 2,
                "texto": texto
            })
        
        if max_respuestas and len(respuestas) >= max_respuestas:
            break
    
    return respuestas, nombre_pregunta

def cargar_catalogo(archivo_catalogo, nombre_pregunta):
    """Carga catálogo histórico de la hoja que coincida con el nombre"""
    try:
        # Intentar cargar el archivo
        excel_file = pd.ExcelFile(archivo_catalogo)
        hojas_disponibles = excel_file.sheet_names
        
        print(f"\n📚 Hojas disponibles en catálogo:")
        for hoja in hojas_disponibles:
            print(f"   • {hoja}")
        
        # Buscar hoja que coincida con la pregunta
        hoja_encontrada = None
        
        # Búsqueda exacta
        if nombre_pregunta in hojas_disponibles:
            hoja_encontrada = nombre_pregunta
        else:
            # Búsqueda flexible (normalizar nombres)
            nombre_norm = nombre_pregunta.lower().strip()
            for hoja in hojas_disponibles:
                hoja_norm = hoja.lower().strip()
                if nombre_norm in hoja_norm or hoja_norm in nombre_norm:
                    hoja_encontrada = hoja
                    break
        
        if not hoja_encontrada:
            print(f"\n⚠️  No se encontró hoja para: '{nombre_pregunta}'")
            return []
        
        print(f"\n✅ Usando hoja: '{hoja_encontrada}'")
        
        # Cargar catálogo
        df = pd.read_excel(archivo_catalogo, sheet_name=hoja_encontrada)
        
        if 'COD' not in df.columns or 'TEXTO' not in df.columns:
            print(f"⚠️  Hoja sin columnas COD/TEXTO")
            return []
        
        catalogo = []
        for _, row in df.iterrows():
            if pd.notna(row['COD']) and pd.notna(row['TEXTO']):
                codigo = int(row['COD'])
                # Evitar duplicar códigos especiales
                if codigo not in [c["codigo"] for c in CODIGOS_ESPECIALES]:
                    catalogo.append({
                        "codigo": codigo,
                        "descripcion": str(row['TEXTO']).strip()
                    })
        
        # SIEMPRE agregar códigos especiales al final
        catalogo.extend(CODIGOS_ESPECIALES)
        
        print(f"✅ Catálogo cargado: {len(catalogo)} códigos (incluye {len(CODIGOS_ESPECIALES)} códigos especiales)")
        return catalogo
        
    except Exception as e:
        print(f"\n❌ Error al cargar catálogo: {str(e)}")
        return []

# Cargar datos
respuestas_reales, nombre_pregunta = cargar_respuestas(ARCHIVO_RESPUESTAS, MAX_RESPUESTAS)
print(f"✅ Respuestas cargadas: {len(respuestas_reales)}")
print(f"✅ Pregunta: {nombre_pregunta}")

if USAR_CATALOGO_HISTORICO:
    catalogo_historico = cargar_catalogo(ARCHIVO_CATALOGO, nombre_pregunta)
    print(f"✅ Catálogo: {len(catalogo_historico)} códigos")
else:
    catalogo_historico = []
    print("⚠️  Sin catálogo histórico")

# SIEMPRE agregar códigos especiales al catálogo (aunque no haya catálogo histórico)
# Evitar duplicados si ya están en el catálogo
codigos_existentes = {c["codigo"] for c in catalogo_historico}
for cod_esp in CODIGOS_ESPECIALES:
    if cod_esp["codigo"] not in codigos_existentes:
        catalogo_historico.append(cod_esp.copy())
        codigos_existentes.add(cod_esp["codigo"])

print(f"✅ Catálogo final: {len(catalogo_historico)} códigos (incluye códigos especiales 90-98)")

✅ Respuestas cargadas: 14
✅ Pregunta: P2
⚠️  Sin catálogo histórico
✅ Catálogo final: 9 códigos (incluye códigos especiales 90-98)


In [20]:
# ESQUEMAS PYDANTIC V3

# 1️⃣ Validación
class ValidacionRespuesta(BaseModel):
    respuesta_id: int
    es_valida: bool
    razon: str

class ResultadoValidacion(BaseModel):
    validaciones: List[ValidacionRespuesta]

# 2️⃣ Evaluación Booleana (NUEVO)
class EvaluacionCodigo(BaseModel):
    codigo: int
    aplica: bool  # True/False explícito
    confianza: float = Field(ge=0.0, le=1.0)

class EvaluacionCatalogo(BaseModel):
    respuesta_id: int
    evaluaciones: List[EvaluacionCodigo]

class ResultadoEvaluacion(BaseModel):
    evaluaciones: List[EvaluacionCatalogo]

# 3️⃣ Análisis de Cobertura (NUEVO)
class ConceptoNuevo(BaseModel):
    codigo: int  # ID numérico secuencial
    descripcion: str
    texto_original: str

class AnalisisCobertura(BaseModel):
    respuesta_id: int
    respuesta_cubierta_completamente: bool
    conceptos_nuevos: List[ConceptoNuevo] = Field(default_factory=list)

class ResultadoCobertura(BaseModel):
    analisis: List[AnalisisCobertura]

# 4️⃣ Justificación
class Justificacion(BaseModel):
    respuesta_id: int
    justificacion: str

class ResultadoJustificacion(BaseModel):
    justificaciones: List[Justificacion]

print("✅ Esquemas Pydantic V3 definidos")

✅ Esquemas Pydantic V3 definidos


In [21]:
# NODO 1: VALIDAR (con detección temprana de códigos especiales)
def nodo_validar(state):
    inicio_tiempo = time.time()
    print(f"\n✅ Validando {len(state['batch_respuestas'])} respuestas...")
    
    # 🆕 MEJORA: Detección temprana de códigos especiales
    respuestas_para_validar = []
    respuestas_especiales = {}  # respuesta_id -> codigo_especial
    
    for i, resp in enumerate(state["batch_respuestas"]):
        codigo_especial = detectar_codigo_especial(resp["texto"])
        if codigo_especial:
            respuestas_especiales[i + 1] = codigo_especial
            # Actualizar métricas
            if "metricas" in state:
                state["metricas"]["respuestas_especiales_detectadas"] += 1
            print(f"   🎯 Respuesta {i+1} detectada como código especial {codigo_especial}")
        else:
            respuestas_para_validar.append((i, resp))
    
    # Si todas son especiales, crear validaciones directamente
    if not respuestas_para_validar:
        validaciones = []
        for i, resp in enumerate(state["batch_respuestas"]):
            codigo_esp = respuestas_especiales.get(i + 1)
            validaciones.append({
                "respuesta_id": i + 1,
                "es_valida": True,  # Las especiales son válidas
                "razon": f"Código especial {codigo_esp} detectado automáticamente"
            })
        
        tiempo_nodo = time.time() - inicio_tiempo
        if "metricas" in state:
            registrar_tiempo_nodo(state["metricas"], "validar", tiempo_nodo)
        
        return {
            **state,
            "validaciones_batch": validaciones,
            "respuestas_especiales": respuestas_especiales  # Guardar para usar después
        }
    
    # Validar las que no son especiales con LLM
    respuestas_str = "\n".join([
        f"{i+1}. {r['texto']}"
        for i, r in respuestas_para_validar
    ])
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Eres un experto en filtrar respuestas de encuestas.

RECHAZAR si:
- Está vacía o solo tiene \"-\" o \".\"
- Es incomprensible
- No responde a la pregunta

ACEPTAR si tiene contenido relevante. Responde en JSON."""),
        ("user", "PREGUNTA: {pregunta}\n\nRESPUESTAS:\n{respuestas}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoValidacion)
    
    # 🆕 MEJORA: Usar retry
    try:
        if "metricas" in state:
            state["metricas"]["llamadas_llm"] += 1
        resultado = llamada_llm_con_retry(chain, {
            "pregunta": state["pregunta"],
            "respuestas": respuestas_str
        })
    except Exception as e:
        if "metricas" in state:
            state["metricas"]["errores_llm"] += 1
        print(f"   ❌ Error en validación: {e}")
        raise
    
    # Combinar validaciones: especiales + LLM
    validaciones = []
    idx_llm = 0
    for i, resp in enumerate(state["batch_respuestas"]):
        if (i + 1) in respuestas_especiales:
            validaciones.append({
                "respuesta_id": i + 1,
                "es_valida": True,
                "razon": f"Código especial {respuestas_especiales[i+1]} detectado automáticamente"
            })
        else:
            validaciones.append(resultado.validaciones[idx_llm].model_dump())
            idx_llm += 1
    
    validas = sum(1 for v in validaciones if v["es_valida"])
    print(f"   ✅ Válidas: {validas}/{len(validaciones)}")
    
    tiempo_nodo = time.time() - inicio_tiempo
    if "metricas" in state:
        registrar_tiempo_nodo(state["metricas"], "validar", tiempo_nodo)
    
    return {
        **state,
        "validaciones_batch": validaciones,
        "respuestas_especiales": respuestas_especiales
    }

print("✅ Nodo validar definido")

✅ Nodo validar definido


In [22]:
# NODO 2: EVALUAR_CATALOGO (BOOLEANO EXHAUSTIVO)
def nodo_evaluar_catalogo_v3(state):
    validas = [
        (i, resp) 
        for i, (resp, val) in enumerate(zip(
            state["batch_respuestas"],
            state["validaciones_batch"]
        ))
        if val["es_valida"]
    ]
    
    if not validas or not state["catalogo"]:
        print(f"   ⚠️  Sin respuestas válidas o sin catálogo")
        return {**state, "evaluaciones_batch": []}
    
    print(f"\n📊 Evaluando catálogo para {len(validas)} respuestas...")
    
    respuestas_str = "\n".join([f"{idx+1}. {resp['texto']}" for idx, resp in validas])
    
    # Separar códigos especiales del resto
    codigos_normales = [c for c in state["catalogo"] if c["codigo"] < 90]
    codigos_especiales = [c for c in state["catalogo"] if c["codigo"] >= 90]
    
    # Limitar códigos normales a 30 para no saturar el prompt
    catalogo_str = "\n".join([
        f"  {c['codigo']}. {c['descripcion']}"
        for c in codigos_normales[:30]
    ])
    
    # Siempre incluir códigos especiales completos
    if codigos_especiales:
        catalogo_str += "\n\n**CÓDIGOS ESPECIALES (90-98):**\n"
        catalogo_str += "\n".join([
            f"  {c['codigo']}. {c['descripcion']}"
            for c in codigos_especiales
        ])
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""Eres un experto en codificación de respuestas de encuestas.

CATÁLOGO HISTÓRICO:
{catalogo_str}

**TU TAREA:**
Para CADA respuesta, evalúa CADA código del catálogo:
- aplica = True: Si el código describe el contenido de la respuesta
- aplica = False: Si NO es relevante
- confianza: 0.0 a 1.0 (qué tan seguro estás)

**REGLAS DE EVALUACIÓN:**

1. **Considera el nivel de especificidad del código:**
   - Los códigos son GENERALES pero CLAROS
   - Ejemplo: Si el código dice "Apto para diabetes", aplica a respuestas que mencionen diabetes, endulzante para diabéticos, etc.
   - NO busques coincidencias exactas de palabras, busca la IDEA CENTRAL

2. **Una respuesta puede tener múltiples códigos aplicables:**
   - Si la respuesta toca varios temas del catálogo, marca aplica=True para TODOS los relevantes
   - Ejemplo: "Es apto para diabetes y no tiene calorías" → aplica=True para "Apto para diabetes" Y "Sin calorías"

3. **Confianza:**
   - 0.95-1.0: Coincidencia clara y directa (SOLO estos se aplicarán)
   - 0.85-0.94: Coincidencia probable pero con alguna variación (NO se aplicarán)
   - <0.85: No aplica (marca aplica=False)

4. **Sé MUY CONSERVADOR:**
   - Solo marca aplica=True si estás MUY seguro (confianza >= 0.95)
   - Precisión > Cobertura: Mejor dejar sin código que asignar incorrecto
   - El umbral de confianza es ALTO (0.95) para asegurar asignaciones correctas

5. **CÓDIGOS ESPECIALES (90-98):**
   - Usa estos códigos cuando la respuesta indique:
     * 90 "Ninguno": La respuesta dice "ninguno", "ninguna", "nada de eso"
     * 91 "No Recuerda": La respuesta dice "no recuerdo", "no me acuerdo"
     * 92 "No Sabe": La respuesta dice "no sé", "no conozco", "no tengo idea"
     * 93 "No Responde": La respuesta está vacía, es "-", ".", o no responde
     * 94 "Cualquiera": La respuesta dice "cualquiera", "cualquier", "da igual"
     * 95 "Todos": La respuesta dice "todos", "todas", "todos los"
     * 96 "No Aplica": La respuesta indica que la pregunta no aplica a su caso
     * 97 "Ningún Otro": La respuesta dice "ningún otro", "ninguna otra"
     * 98 "Nada": La respuesta dice "nada", "nada más", "nada en particular"
   - Estos códigos pueden aplicarse SOLOS o junto con otros códigos según el contexto

Evalúa TODOS los códigos para TODAS las respuestas.
Responde en JSON."""),
        ("user", "PREGUNTA: {pregunta}\n\nRESPUESTAS:\n{respuestas}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoEvaluacion)
    
    # 🆕 MEJORA: Usar retry y métricas
    inicio_tiempo = time.time()
    try:
        if "metricas" in state:
            state["metricas"]["llamadas_llm"] += 1
        resultado = llamada_llm_con_retry(chain, {
            "pregunta": state["pregunta"],
            "respuestas": respuestas_str
        })
    except Exception as e:
        if "metricas" in state:
            state["metricas"]["errores_llm"] += 1
        print(f"   ❌ Error en evaluación: {e}")
        raise
    
    matches = sum(
        1 for ev in resultado.evaluaciones
        for cod in ev.evaluaciones
        if cod.aplica and cod.confianza >= 0.95
    )
    
    print(f"   ✅ Matches (confianza >= 0.95): {matches}")
    
    tiempo_nodo = time.time() - inicio_tiempo
    if "metricas" in state:
        registrar_tiempo_nodo(state["metricas"], "evaluar_catalogo", tiempo_nodo)
    
    return {
        **state,
        "evaluaciones_batch": [ev.model_dump() for ev in resultado.evaluaciones]
    }

print("✅ Nodo evaluar_catalogo_v3 definido")

✅ Nodo evaluar_catalogo_v3 definido


In [23]:
# NODO 3: IDENTIFICAR_CONCEPTOS_NUEVOS (DETECTAR GAPS)
def nodo_identificar_conceptos_v3(state):
    # Obtener TODAS las respuestas válidas (con o sin catálogo)
    validas_con_eval = []
    
    for i, (resp, val) in enumerate(zip(
        state["batch_respuestas"],
        state["validaciones_batch"]
    )):
        if not val["es_valida"]:
            continue
        
        resp_id = i + 1
        
        # Buscar evaluación (puede estar vacío si no hay catálogo)
        evaluacion = next(
            (ev for ev in state["evaluaciones_batch"] if ev["respuesta_id"] == resp_id),
            None
        )
        
        if evaluacion:
            # Hay catálogo: obtener códigos aplicados (solo con confianza >= 0.95)
            codigos_aplicados = [
                cod["codigo"] for cod in evaluacion["evaluaciones"]
                if cod["aplica"] and cod["confianza"] >= 0.95
            ]
        else:
            # NO hay catálogo o no hay evaluación: códigos_asignados = []
            codigos_aplicados = []
        
        # SIEMPRE agregar la respuesta válida (con o sin códigos históricos)
        validas_con_eval.append({
            "respuesta_id": resp_id,
            "texto": resp["texto"],
            "codigos_asignados": codigos_aplicados
        })
    
    if not validas_con_eval:
        print(f"   ⚠️  Sin respuestas válidas")
        return {**state, "cobertura_batch": []}
    
    # Usar el contador global de códigos nuevos
    codigo_base = state["proximo_codigo_nuevo"]
    
    print(f"\n🔍 Analizando cobertura para {len(validas_con_eval)} respuestas...")
    print(f"   (Códigos nuevos empezarán desde: {codigo_base})")
    
    # Recolectar códigos ya creados en batches anteriores para evitar duplicados
    codigos_ya_creados = {}
    if state.get("codificaciones"):
        for cod in state["codificaciones"]:
            if cod.get("codigos_nuevos"):
                for nuevo in cod["codigos_nuevos"]:
                    cod_id = nuevo.get("codigo")
                    desc = nuevo.get("descripcion", "")
                    if cod_id and desc:
                        codigos_ya_creados[cod_id] = desc
    
    # Construir string de códigos ya creados para el prompt
    codigos_existentes_str = ""
    if codigos_ya_creados:
        codigos_existentes_str = "\n**CÓDIGOS NUEVOS YA CREADOS EN BATCHES ANTERIORES:**\n"
        for cod_id in sorted(codigos_ya_creados.keys()):
            codigos_existentes_str += f"  {cod_id}: {codigos_ya_creados[cod_id]}\n"
        codigos_existentes_str += "\n**IMPORTANTE:** Si encuentras un concepto similar a uno de estos, NO crees un código nuevo. El sistema los agrupará después.\n"
    
    respuestas_str = ""
    for item in validas_con_eval:
        resp_id = item["respuesta_id"]
        texto = item["texto"]
        codigos = item["codigos_asignados"]
        
        if codigos and state["catalogo"]:
            # Hay códigos asignados: mostrar descripciones
            descrips = []
            for cod_id in codigos:
                desc = next(
                    (c["descripcion"] for c in state["catalogo"] if c["codigo"] == cod_id),
                    f"Código {cod_id}"
                )
                descrips.append(f"[{cod_id}: {desc}]")
            
            respuestas_str += f"{resp_id}. \"{texto}\"\n   Códigos asignados: {', '.join(descrips)}\n\n"
        else:
            # NO hay códigos: toda la respuesta necesita códigos nuevos
            respuestas_str += f"{resp_id}. \"{texto}\"\n   Códigos asignados: NINGUNO (generar códigos para TODA la respuesta)\n\n"
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""Eres un experto en codificación de respuestas de encuestas de opinión pública.

**PREGUNTA DE LA ENCUESTA:**
{{pregunta}}

**PROCESO DE TRABAJO - SEGUIR EN ESTE ORDEN:**

**PASO 1: LEE TODAS LAS RESPUESTAS PRIMERO**
- Antes de crear cualquier código, lee y analiza TODAS las respuestas del batch
- Identifica los conceptos únicos que aparecen en todas las respuestas
- NO crees códigos de forma aislada para cada respuesta
- Compara conceptos entre respuestas para identificar duplicados

**PASO 2: IDENTIFICA CONCEPTOS ÚNICOS Y AGRÚPALOS**
- Agrupa respuestas que mencionan el mismo concepto
- Ejemplo: Si varias respuestas mencionan "saludable", "es saludable", "muy saludable" → Es el MISMO concepto → UN SOLO código
- Ejemplo: Si varias respuestas mencionan "apto para diabetes", "para diabéticos", "endulzante para personas con diabetes" → Es el MISMO concepto → UN SOLO código
- **CRÍTICO:** Si ya identificaste un concepto en una respuesta anterior, REUTILIZA ese mismo código para respuestas similares

**PASO 3: CREA CÓDIGOS CON REDACCIÓN COHERENTE Y ÚNICA**
- Asigna UN código a cada concepto único identificado
- La redacción debe ser COHERENTE: no muy general, no muy específica
- ✅ CORRECTO: "Saludable", "Apto para diabetes", "Sin calorías", "Versatilidad de uso"
- ❌ MUY GENERAL: "Bueno", "Útil", "Me gusta"
- ❌ MUY ESPECÍFICO: "Saludable para personas con diabetes tipo 2 que buscan endulzantes naturales", "Versatilidad de uso en comidas y bebidas calientes"

**REGLAS CRÍTICAS DE ESPECIFICIDAD Y UNICIDAD:**

1. **Precisión > Cobertura** (mejor dejar sin código que asignar incorrecto)

2. **Nivel de especificidad - CRÍTICO:**
   - ✅ CORRECTO: "Versatilidad de uso", "Apto para diabetes", "Sin calorías", "Sabor", "Textura"
   - ❌ MUY GENERAL: "Bueno", "Útil", "Me gusta", "Calidad"
   - ❌ MUY ESPECÍFICO: "Versatilidad de uso en comidas", "Apto para personas con diabetes tipo 2", "Sabor dulce natural sin químicos"
   - **Principio:** Si dos descripciones comparten la MISMA IDEA CENTRAL, deben usar el MISMO código

3. **Agrupa bajo el MISMO código si:**
   - Comparten el tema/concepto principal
   - Solo difieren en detalles o contexto específico
   - Ejemplo: "Sabor agradable", "Buen sabor", "Sabor rico" → MISMO código "Sabor"
   - Ejemplo: "Apto para diabetes", "Para diabéticos", "Endulzante para personas con diabetes" → MISMO código "Apto para diabetes"

4. **Crea CÓDIGOS SEPARADOS solo si:**
   - Son temas REALMENTE distintos e independientes
   - No se pueden agrupar bajo una categoría común
   - Ejemplo: "Sabor" vs "Textura" vs "Precio" → Diferentes códigos
   - Ejemplo: "Apto para diabetes" vs "Sin calorías" vs "Versatilidad de uso" → Diferentes códigos

5. **Descripciones GENERALES pero CLARAS:**
   - ✅ BIEN: "Precio accesible", "Sabor", "Textura", "Calidad nutricional", "Apto para diabetes", "Sin calorías"
   - ❌ MAL: "Precio accesible para familias", "Sabor dulce natural", "Textura suave", "Apto para personas con diabetes tipo 2"
   - Usa el nivel de abstracción del catálogo histórico como referencia (si existe)

6. **NO uses frases como:** "Mención sobre...", "Referencias a...", "Menciones de...", "Percepción de..."

7. **UNICIDAD Y REUTILIZACIÓN - CRÍTICO:**
   - Cada código = Un concepto único
   - Si encuentras el mismo concepto en múltiples respuestas, REUTILIZA el mismo código
   - NO crees códigos diferentes para el mismo concepto con diferentes redacciones
   - **Ejemplo PROHIBIDO:** NO crees código {codigo_base} para "Saludable" y luego código {codigo_base + 1} para "Es saludable" o código {codigo_base + 2} para "Muy saludable"
   - **TODOS deben usar el MISMO código {codigo_base} con la MISMA descripción "Saludable"**

8. **COHERENCIA EN LA REDACCIÓN:**
   - Si ya creaste un código para un concepto, usa la MISMA redacción para ese concepto en todas las respuestas
   - NO crees códigos diferentes con redacciones diferentes para el mismo concepto
   - Mantén consistencia: misma descripción = mismo código

**FORMATO DE CÓDIGOS:**
- codigo: Número secuencial ÚNICO empezando desde {codigo_base}
- descripcion: Descripción COHERENTE (equilibrio entre general y específico)
- texto_original: Fragmento del texto que justifica el código

**SECUENCIALIDAD:**
- Primer código nuevo: {codigo_base}
- Segundo código nuevo: {codigo_base + 1}
- Tercer código nuevo: {codigo_base + 2}
- etc.

**DECISIONES:**
- Si NO hay códigos asignados: respuesta_cubierta_completamente=False, generar códigos
- Si hay códigos pero faltan conceptos: respuesta_cubierta_completamente=False, agregar códigos
- Si está completamente cubierta: respuesta_cubierta_completamente=True, conceptos_nuevos=[]
- Puedes generar MÚLTIPLES códigos por respuesta si toca temas distintos

**RECORDATORIO CRÍTICO - LEE ANTES DE RESPONDER:**
1. LEE TODAS LAS RESPUESTAS PRIMERO (no crees códigos aislados)
2. IDENTIFICA CONCEPTOS ÚNICOS (agrupa similares bajo el mismo código)
3. CREA UN CÓDIGO POR CONCEPTO ÚNICO (no repitas códigos con el mismo texto)
4. REUTILIZA EL MISMO CÓDIGO para el mismo concepto en diferentes respuestas
5. MANTÉN COHERENCIA en la redacción (misma descripción para el mismo concepto)
6. NO crees códigos diferentes para variaciones del mismo concepto (ej: "Saludable", "Es saludable", "Muy saludable" → TODOS el mismo código)

Responde en JSON."""),
        ("user", "PREGUNTA: {pregunta}\n{codigos_existentes}RESPUESTAS:\n{respuestas}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoCobertura)
    
    # 🆕 MEJORA: Usar retry y métricas
    inicio_tiempo = time.time()
    try:
        if "metricas" in state:
            state["metricas"]["llamadas_llm"] += 1
        resultado = llamada_llm_con_retry(chain, {
            "pregunta": state["pregunta"],
            "codigos_existentes": codigos_existentes_str,
            "respuestas": respuestas_str
        })
    except Exception as e:
        if "metricas" in state:
            state["metricas"]["errores_llm"] += 1
        print(f"   ❌ Error en identificación: {e}")
        raise
    
    # 🆕 MEJORA: Deduplicación automática como respaldo (solo si el modelo no lo hizo bien)
    # Recolectar TODOS los conceptos del batch
    todos_conceptos_batch = []
    for analisis in resultado.analisis:
        for concepto in analisis.conceptos_nuevos:
            concepto_dict = concepto.model_dump()
            concepto_dict["_respuesta_id"] = analisis.respuesta_id
            todos_conceptos_batch.append(concepto_dict)
    
    # Detectar duplicados obvios (descripciones idénticas o muy similares)
    grupos_similares = {}  # descripcion_normalizada -> lista de conceptos
    for concepto in todos_conceptos_batch:
        desc = concepto.get("descripcion", "")
        desc_norm = normalizar_texto(desc)
        
        # Buscar si ya existe un grupo similar
        encontrado = False
        for desc_existente, grupo in grupos_similares.items():
            if son_conceptos_similares(desc, desc_existente):
                grupo.append(concepto)
                encontrado = True
                break
        
        if not encontrado:
            grupos_similares[desc_norm] = [concepto]
    
    # Crear mapeo solo para duplicados obvios (descripciones idénticas o casi idénticas)
    mapeo_deduplicacion = {}  # descripcion_normalizada -> codigo_unificado
    codigo_unificado = codigo_base
    descripciones_finales = {}  # codigo_unificado -> descripcion_final
    
    duplicados_detectados = 0
    for desc_norm, grupo in grupos_similares.items():
        # Solo agrupar si hay más de un concepto (duplicado obvio)
        if len(grupo) > 1:
            # Usar la descripción más corta como representante
            desc_final = min(grupo, key=lambda x: len(x.get("descripcion", ""))).get("descripcion", "")
            mapeo_deduplicacion[desc_norm] = codigo_unificado
            descripciones_finales[codigo_unificado] = desc_final
            
            duplicados_detectados += len(grupo) - 1
            descs_originales = [c.get("descripcion", "") for c in grupo]
            print(f"   🔗 Agrupando {len(grupo)} conceptos similares bajo código {codigo_unificado}: {desc_final}")
            print(f"      Originales: {', '.join(set(descs_originales))}")
            
            codigo_unificado += 1
        else:
            # Concepto único, mantener su código original
            concepto = grupo[0]
            cod_orig = concepto.get("codigo", codigo_unificado)
            mapeo_deduplicacion[desc_norm] = cod_orig
            descripciones_finales[cod_orig] = concepto.get("descripcion", "")
            codigo_unificado = max(codigo_unificado, cod_orig + 1)
    
    if duplicados_detectados > 0:
        print(f"   ✅ Duplicados detectados y agrupados: {duplicados_detectados}")
    
    # Aplicar deduplicación solo a duplicados obvios
    analisis_corregidos = []
    codigo_actual = codigo_base

    for analisis in resultado.analisis:
        conceptos_nuevos = [c.model_dump() for c in analisis.conceptos_nuevos]

        # Aplicar deduplicación solo si hay duplicados obvios
        conceptos_dedup = []
        codigos_vistos = set()

        for concepto in conceptos_nuevos:
            desc = concepto.get("descripcion", "")
            desc_norm = normalizar_texto(desc)

            # Solo aplicar deduplicación si está en el mapeo (duplicado detectado)
            if desc_norm in mapeo_deduplicacion:
                codigo_final = mapeo_deduplicacion[desc_norm]
                desc_final = descripciones_finales.get(codigo_final, desc)
            else:
                # Mantener código original del modelo
                codigo_final = concepto.get("codigo", codigo_actual)
                desc_final = desc

            # Evitar duplicados en la misma respuesta
            if codigo_final not in codigos_vistos:
                codigos_vistos.add(codigo_final)
                concepto_final = {
                    "codigo": codigo_final,
                    "descripcion": desc_final,
                    "texto_original": concepto.get("texto_original", "")
                }
                conceptos_dedup.append(concepto_final)

        # NO volver a renumerar aquí: usamos directamente conceptos_dedup
        analisis_corregidos.append({
            "respuesta_id": analisis.respuesta_id,
            "respuesta_cubierta_completamente": analisis.respuesta_cubierta_completamente,
            "conceptos_nuevos": conceptos_dedup
        })

    # Actualizar el siguiente código global: tomar el máximo código usado en el batch + 1
    if todos_conceptos_batch:
        max_codigo = max(
            c["codigo"] for c in todos_conceptos_batch
            if c.get("codigo") is not None
        )
        codigo_actual = max(max_codigo + 1, codigo_base)

    cubiertas = sum(1 for a in analisis_corregidos if a["respuesta_cubierta_completamente"])
    con_nuevos = sum(1 for a in analisis_corregidos if len(a["conceptos_nuevos"]) > 0)
    total_conceptos = sum(len(a["conceptos_nuevos"]) for a in analisis_corregidos)

    print(f"   ✅ Completamente cubiertas: {cubiertas}/{len(analisis_corregidos)}")
    print(f"   🆕 Con conceptos nuevos: {con_nuevos}/{len(analisis_corregidos)}")
    print(f"   🆕 Total conceptos nuevos: {total_conceptos}")

    tiempo_nodo = time.time() - inicio_tiempo
    if "metricas" in state:
        registrar_tiempo_nodo(state["metricas"], "identificar_conceptos", tiempo_nodo)

    return {
        **state,
        "cobertura_batch": analisis_corregidos,
        "proximo_codigo_nuevo": codigo_actual  # Actualizar para el siguiente batch
    }

def procesar_grupo_respuestas(respuestas_grupo, state, codigo_base, codigos_historicos_categoria, categoria):
    """
    Procesa un grupo de respuestas (con o sin categoría)
    
    Args:
        respuestas_grupo: Lista de respuestas del mismo grupo
        state: Estado del grafo
        codigo_base: Código base para empezar a generar códigos nuevos
        codigos_historicos_categoria: Códigos históricos relacionados con esta categoría (para referencia)
        categoria: Nombre de la categoría (o None si no hay categoría)
    """
    # Recolectar códigos ya creados en batches anteriores para esta categoría
    codigos_ya_creados = {}
    if state.get("codificaciones"):
        for cod in state["codificaciones"]:
            # Solo incluir códigos de la misma categoría
            if categoria and cod.get("dato_auxiliar") != categoria:
                continue
            if cod.get("codigos_nuevos"):
                for nuevo in cod["codigos_nuevos"]:
                    cod_id = nuevo.get("codigo")
                    desc = nuevo.get("descripcion", "")
                    if cod_id and desc:
                        codigos_ya_creados[cod_id] = desc
    
    # Construir string de códigos ya creados
    codigos_existentes_str = ""
    if codigos_ya_creados:
        codigos_existentes_str = "\n**CÓDIGOS NUEVOS YA CREADOS EN BATCHES ANTERIORES:**\n"
        for cod_id in sorted(codigos_ya_creados.keys()):
            codigos_existentes_str += f"  {cod_id}: {codigos_ya_creados[cod_id]}\n"
        codigos_existentes_str += "\n**IMPORTANTE:** Si encuentras un concepto similar a uno de estos, NO crees un código nuevo. El sistema los agrupará después.\n"
    
    # Construir string de respuestas
    respuestas_str = ""
    for item in respuestas_grupo:
        resp_id = item["respuesta_id"]
        texto = item["texto"]
        codigos = item["codigos_asignados"]
        
        if codigos and state["catalogo"]:
            descrips = []
            for cod_id in codigos:
                desc = next(
                    (c["descripcion"] for c in state["catalogo"] if c["codigo"] == cod_id),
                    f"Código {cod_id}"
                )
                descrips.append(f"[{cod_id}: {desc}]")
            respuestas_str += f"{resp_id}. \"{texto}\"\n   Códigos asignados: {', '.join(descrips)}\n\n"
        else:
            respuestas_str += f"{resp_id}. \"{texto}\"\n   Códigos asignados: NINGUNO (generar códigos para TODA la respuesta)\n\n"
    
    # Construir prompt con información de categoría si hay
    info_categoria = ""
    if categoria:
        info_categoria = f"""
**INFORMACIÓN DE CATEGORÍA:**
- Esta respuesta pertenece a la categoría: **{categoria}**
- Los códigos nuevos para esta categoría empezarán desde: **{codigo_base}**
- Genera códigos secuenciales dentro de esta categoría: {codigo_base}, {codigo_base + 1}, {codigo_base + 2}, etc.
"""
        if codigos_historicos_categoria:
            info_categoria += f"- Hay {len(codigos_historicos_categoria)} códigos históricos relacionados con esta categoría (para referencia)\n"
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""Eres un experto en codificación de respuestas de encuestas de opinión pública.

Tu trabajo:
1. Si NO hay códigos asignados → Crear códigos para TODA la respuesta
2. Si hay códigos asignados → Identificar conceptos NO CUBIERTOS
{info_categoria}
**REGLAS CRÍTICAS DE ESPECIFICIDAD:**

1. **Nivel de especificidad - CRÍTICO:**
   - ✅ CORRECTO: "Versatilidad de uso", "Apto para diabetes", "Sin calorías"
   - ❌ INCORRECTO: "Versatilidad de uso en comidas", "Apto para personas con diabetes tipo 2", "Sin calorías adicionales al consumirlo"
   - Principio: Si dos descripciones comparten la MISMA IDEA CENTRAL, deben usar el MISMO código

2. **Agrupa bajo el MISMO código si:**
   - Comparten el tema/concepto principal
   - Solo difieren en detalles o contexto específico
   - Ejemplo: "Sabor agradable", "Buen sabor", "Sabor rico" → MISMO código "Sabor"
   - Ejemplo: "Apto para diabetes", "Para diabéticos", "Endulzante para personas con diabetes" → MISMO código "Apto para diabetes"

3. **Crea CÓDIGOS SEPARADOS solo si:**
   - Son temas REALMENTE distintos e independientes
   - No se pueden agrupar bajo una categoría común
   - Ejemplo: "Sabor" vs "Textura" vs "Precio" → Diferentes códigos
   - Ejemplo: "Apto para diabetes" vs "Sin calorías" vs "Versatilidad de uso" → Diferentes códigos

4. **Descripciones GENERALES pero CLARAS:**
   - ✅ BIEN: "Precio accesible", "Sabor", "Textura", "Calidad nutricional", "Apto para diabetes", "Sin calorías"
   - ❌ MAL: "Precio accesible para familias", "Sabor dulce natural", "Textura suave", "Apto para personas con diabetes tipo 2"

5. **NO uses frases como:** "Mención sobre...", "Referencias a...", "Menciones de...", "Percepción de..."

6. **CADA código debe ser ÚNICO:**
   - Un código = Un concepto específico
   - NO reutilices el mismo código para conceptos diferentes
   - Si encuentras un concepto similar a uno ya creado, REUTILIZA ese código

**FORMATO DE CÓDIGOS:**
- codigo: Número secuencial ÚNICO empezando desde {codigo_base}
- descripcion: Descripción GENERAL pero CLARA (sin detalles específicos)
- texto_original: Fragmento del texto que justifica el código

**SECUENCIALIDAD:**
- Primer código nuevo: {codigo_base}
- Segundo código nuevo: {codigo_base + 1}
- Tercer código nuevo: {codigo_base + 2}
- etc.

**DECISIONES:**
- Si NO hay códigos asignados: respuesta_cubierta_completamente=False, generar códigos
- Si hay códigos pero faltan conceptos: respuesta_cubierta_completamente=False, agregar códigos
- Si está completamente cubierta: respuesta_cubierta_completamente=True, conceptos_nuevos=[]
- Puedes generar MÚLTIPLES códigos por respuesta si toca temas distintos

**IMPORTANTE:** 
- Revisa si un concepto similar ya fue creado en respuestas anteriores del batch
- Si es similar, REUTILIZA el código existente en lugar de crear uno nuevo
- Cada código debe representar UN SOLO concepto único
- NO crees códigos diferentes para variaciones del mismo concepto
- Ejemplo: Si ya creaste código {codigo_base} para "Apto para diabetes", NO crees otro código para "Para diabéticos" o "Endulzante para personas con diabetes"

Responde en JSON."""),
        ("user", "PREGUNTA: {pregunta}\n{codigos_existentes}RESPUESTAS:\n{respuestas}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoCobertura)
    
    resultado = chain.invoke({
        "pregunta": state["pregunta"],
        "codigos_existentes": codigos_existentes_str,
        "respuestas": respuestas_str
    })
    
    # Los códigos generados son secuenciales dentro de la categoría
    # No hay validación de rangos - el sistema confía en que el LLM genere códigos correctos
    # Si hay problemas, se pueden ajustar manualmente después
    
    cubiertas = sum(1 for a in resultado.analisis if a.respuesta_cubierta_completamente)
    con_nuevos = sum(1 for a in resultado.analisis if len(a.conceptos_nuevos) > 0)
    total_conceptos = sum(len(a.conceptos_nuevos) for a in resultado.analisis)
    
    print(f"   ✅ Completamente cubiertas: {cubiertas}/{len(resultado.analisis)}")
    print(f"   🆕 Con conceptos nuevos: {con_nuevos}/{len(resultado.analisis)}")
    print(f"   🆕 Total conceptos nuevos: {total_conceptos}")
    
    return [a.model_dump() for a in resultado.analisis]

print("✅ Nodo identificar_conceptos_v3 definido")

✅ Nodo identificar_conceptos_v3 definido


In [24]:
# NODO 4: JUSTIFICAR
def nodo_justificar_v3(state):
    print(f"\n📝 Generando justificaciones...")
    
    resumen = []
    for i, (resp, val) in enumerate(zip(state["batch_respuestas"], state["validaciones_batch"])):
        resp_id = i + 1
        
        if not val["es_valida"]:
            resumen.append(f"{resp_id}. RECHAZADA: {val['razon']}")
            continue
        
        evaluacion = next((ev for ev in state["evaluaciones_batch"] if ev["respuesta_id"] == resp_id), {"evaluaciones": []})
        codigos_hist = [cod["codigo"] for cod in evaluacion.get("evaluaciones", []) if cod["aplica"] and cod["confianza"] >= 0.95]
        
        cobertura = next((cob for cob in state["cobertura_batch"] if cob["respuesta_id"] == resp_id), {"conceptos_nuevos": []})
        conceptos_nuevos = cobertura.get("conceptos_nuevos", [])
        
        resumen.append(f"{resp_id}. Hist:{len(codigos_hist)} Nuevos:{len(conceptos_nuevos)}")
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Genera justificaciones BREVES (1-2 oraciones).
Sé CONCISO. Responde en JSON."""),
        ("user", "DECISIONES:\n{resumen}")
    ])
    
    llm = ChatOpenAI(model=state["modelo_gpt"], temperature=0)
    chain = prompt | llm.with_structured_output(ResultadoJustificacion)
    
    # 🆕 MEJORA: Usar retry y métricas
    inicio_tiempo = time.time()
    try:
        if "metricas" in state:
            state["metricas"]["llamadas_llm"] += 1
        resultado = llamada_llm_con_retry(chain, {"resumen": "\n".join(resumen)})
    except Exception as e:
        if "metricas" in state:
            state["metricas"]["errores_llm"] += 1
        print(f"   ❌ Error en justificación: {e}")
        raise
    
    print(f"   ✅ Justificaciones generadas")
    
    tiempo_nodo = time.time() - inicio_tiempo
    if "metricas" in state:
        registrar_tiempo_nodo(state["metricas"], "justificar", tiempo_nodo)
    
    return {**state, "justificaciones_batch": [j.model_dump() for j in resultado.justificaciones]}

# NODO 5: ENSAMBLAR
def nodo_ensamblar_v3(state):
    print(f"\n🔧 Ensamblando resultados...")
    
    codificaciones_batch = []
    
    for i, (resp, val) in enumerate(zip(state["batch_respuestas"], state["validaciones_batch"])):
        resp_id = i + 1
        justif = next((j for j in state["justificaciones_batch"] if j["respuesta_id"] == resp_id), {"justificacion": "Sin justificación"})
        
        if not val["es_valida"]:
            codificaciones_batch.append({
                "fila_excel": resp["fila_excel"],
                "texto": resp["texto"],
                "decision": "rechazar",
                "codigos_historicos": [],
                "codigos_nuevos": [],
                "justificacion": justif["justificacion"]
            })
            continue
        
        # 🆕 MEJORA: Verificar si tiene código especial detectado
        codigo_especial = state.get("respuestas_especiales", {}).get(resp_id)
        
        if codigo_especial:
            # Respuesta con código especial detectado automáticamente
            codigos_hist = [codigo_especial]
            codigos_nuevos = []
            decision = "historico"  # Los especiales son históricos
        else:
            evaluacion = next((ev for ev in state["evaluaciones_batch"] if ev["respuesta_id"] == resp_id), {"evaluaciones": []})
            codigos_hist = [cod["codigo"] for cod in evaluacion.get("evaluaciones", []) if cod["aplica"] and cod["confianza"] >= 0.95]
            
            cobertura = next((cob for cob in state["cobertura_batch"] if cob["respuesta_id"] == resp_id), {"conceptos_nuevos": []})
            # Códigos nuevos con ID y descripción
            codigos_nuevos = [
                {"codigo": c["codigo"], "descripcion": c["descripcion"]} 
                for c in cobertura.get("conceptos_nuevos", [])
            ]
            
            if codigos_hist and codigos_nuevos:
                decision = "mixto"
            elif codigos_hist:
                decision = "historico"
            elif codigos_nuevos:
                decision = "nuevo"
            else:
                decision = "rechazar"
        
        codificaciones_batch.append({
            "fila_excel": resp["fila_excel"],
            "texto": resp["texto"],
            "decision": decision,
            "codigos_historicos": codigos_hist,
            "codigos_nuevos": codigos_nuevos,
            "justificacion": justif["justificacion"]
        })
    
    decisiones = {}
    for cod in codificaciones_batch:
        dec = cod["decision"]
        decisiones[dec] = decisiones.get(dec, 0) + 1
    
    print(f"   📊 Decisiones: {decisiones}")
    
    return {
        **state,
        "codificaciones": state["codificaciones"] + codificaciones_batch
    }

print("✅ Nodos justificar_v3 y ensamblar_v3 definidos")

✅ Nodos justificar_v3 y ensamblar_v3 definidos


In [25]:
# NODOS AUXILIARES (reutilizamos del notebook 03)
def nodo_preparar_batch(state):
    inicio = state["batch_actual"] * state["batch_size"]
    fin = inicio + state["batch_size"]
    batch = state["respuestas"][inicio:fin]
    return {**state, "batch_respuestas": batch}

def decidir_continuar(state):
    if (state["batch_actual"] + 1) * state["batch_size"] < len(state["respuestas"]):
        return "preparar_batch"
    return "finalizar"

# Estado
class EstadoCodificacionV3(TypedDict):
    pregunta: str
    modelo_gpt: str
    batch_size: int
    respuestas: List[Dict]
    catalogo: List[Dict]
    batch_actual: int
    batch_respuestas: List[Dict]
    codificaciones: List[Dict]
    validaciones_batch: List[Dict]
    evaluaciones_batch: List[Dict]
    cobertura_batch: List[Dict]
    justificaciones_batch: List[Dict]
    proximo_codigo_nuevo: int  # 🆕 Contador global de códigos nuevos
    metricas: Dict  # 🆕 Métricas de ejecución
    respuestas_especiales: Dict[int, int]  # 🆕 respuesta_id -> codigo_especial

# CONSTRUIR GRAFO V3
workflow_v3 = StateGraph(EstadoCodificacionV3)

workflow_v3.add_node("preparar_batch", nodo_preparar_batch)
workflow_v3.add_node("validar", nodo_validar)
workflow_v3.add_node("evaluar_catalogo", nodo_evaluar_catalogo_v3)
workflow_v3.add_node("identificar_conceptos", nodo_identificar_conceptos_v3)
workflow_v3.add_node("justificar", nodo_justificar_v3)
workflow_v3.add_node("ensamblar", nodo_ensamblar_v3)
workflow_v3.add_node("finalizar", lambda state: {**state, "batch_actual": state["batch_actual"] + 1})

workflow_v3.set_entry_point("preparar_batch")
workflow_v3.add_edge("preparar_batch", "validar")
workflow_v3.add_edge("validar", "evaluar_catalogo")
workflow_v3.add_edge("evaluar_catalogo", "identificar_conceptos")
workflow_v3.add_edge("identificar_conceptos", "justificar")
workflow_v3.add_edge("justificar", "ensamblar")
workflow_v3.add_edge("ensamblar", "finalizar")
workflow_v3.add_conditional_edges(
    "finalizar",
    decidir_continuar,
    {"preparar_batch": "preparar_batch", "finalizar": END}
)

app_v3 = workflow_v3.compile()

print("✅ Grafo V3 compilado")
print("\n📊 Nodos:")
for nodo in app_v3.get_graph().nodes.keys():
    print(f"   • {nodo}")

✅ Grafo V3 compilado

📊 Nodos:
   • __start__
   • preparar_batch
   • validar
   • evaluar_catalogo
   • identificar_conceptos
   • justificar
   • ensamblar
   • finalizar
   • __end__


In [26]:
# EJECUTAR GRAFO V3

# Verificar datos antes de ejecutar
print("="*60)
print("🔍 VERIFICACIÓN PRE-EJECUCIÓN")
print("="*60)
print(f"\n📊 Respuestas cargadas: {len(respuestas_reales)}")
print(f"📚 Catálogo histórico: {len(catalogo_historico)} códigos")
if len(catalogo_historico) == 0:
    print("\n⚠️  ADVERTENCIA: Sin catálogo histórico!")
    print("   Solo se generarán códigos nuevos.")
else:
    print(f"\n✅ Catálogo cargado correctamente")
    print(f"   Primeros 3 códigos:")
    for c in catalogo_historico[:3]:
        print(f"   • [{c['codigo']}] {c['descripcion']}")

# Calcular límite de recursión necesario
batches_esperados = (len(respuestas_reales) + BATCH_SIZE - 1) // BATCH_SIZE
# Cada batch ejecuta ~7 nodos + decisión = ~8 pasos
# Agregar margen de seguridad 20%
recursion_limit = max(batches_esperados * 10, 100)

print(f"\n🧮 Batches esperados: {batches_esperados}")
print(f"⚙️  Límite de recursión: {recursion_limit}")

# Calcular el código inicial para nuevos códigos
# 🆕 CORRECCIÓN: Excluir códigos especiales (90-98) del cálculo
if catalogo_historico:
    # Solo considerar códigos NO especiales (< 90)
    codigos_normales = [c["codigo"] for c in catalogo_historico if c["codigo"] < 90]
    if codigos_normales:
        proximo_codigo_inicial = max(codigos_normales) + 1
    else:
        # Si solo hay códigos especiales, empezar desde 1
        proximo_codigo_inicial = 1
else:
    proximo_codigo_inicial = 1

print(f"🔢 Código inicial para nuevos códigos: {proximo_codigo_inicial}")
print(f"   (Códigos especiales 90-98 excluidos del cálculo)")

# 🆕 MEJORA: Inicializar métricas
metricas = inicializar_metricas()

estado_inicial = {
    "pregunta": nombre_pregunta,
    "modelo_gpt": MODELO_GPT,
    "batch_size": BATCH_SIZE,
    "respuestas": respuestas_reales,
    "catalogo": catalogo_historico,
    "batch_actual": 0,
    "batch_respuestas": [],
    "codificaciones": [],
    "validaciones_batch": [],
    "evaluaciones_batch": [],
    "cobertura_batch": [],
    "justificaciones_batch": [],
    "proximo_codigo_nuevo": proximo_codigo_inicial,
    "metricas": metricas,  # 🆕 Métricas
    "respuestas_especiales": {}  # 🆕 Respuestas especiales detectadas
}

print(f"\n{'='*60}")
print("🚀 EJECUTANDO GRAFO V3")
print("="*60)
print(f"\n📊 Respuestas: {len(respuestas_reales)}")
print(f"📦 Batch size: {BATCH_SIZE}")
print(f"🤖 Modelo: {MODELO_GPT}")
print(f"\n{'='*60}\n")

# EJECUTAR con recursion_limit configurado
from langgraph.pregel.main import RunnableConfig

config = RunnableConfig(recursion_limit=recursion_limit)
resultado_final_v3 = app_v3.invoke(estado_inicial, config=config)

print("\n" + "="*60)
print("✅ PROCESO V3 COMPLETADO")
print("="*60)

# Estadísticas
decisiones = {}
for cod in resultado_final_v3["codificaciones"]:
    dec = cod["decision"]
    decisiones[dec] = decisiones.get(dec, 0) + 1

print(f"\n📊 Resultados:")
print(f"   Total respuestas: {len(resultado_final_v3['codificaciones'])}")
print(f"\n📈 Decisiones:")
for decision, cantidad in sorted(decisiones.items(), key=lambda x: -x[1]):
    porcentaje = (cantidad / len(resultado_final_v3['codificaciones'])) * 100
    print(f"   {decision:12} : {cantidad:3} ({porcentaje:.1f}%)")

# Casos mixtos
mixtos = [c for c in resultado_final_v3["codificaciones"] if c["decision"] == "mixto"]
if mixtos:
    print(f"\n🎯 Casos MIXTOS capturados: {len(mixtos)}")
    print(f"   (Respuestas con códigos históricos Y nuevos)")

# Códigos nuevos generados
nuevos_con_codigos = [c for c in resultado_final_v3["codificaciones"] if c["codigos_nuevos"]]
if nuevos_con_codigos:
    # Recolectar todos los códigos únicos
    codigos_unicos = {}
    for cod in nuevos_con_codigos:
        for nuevo in cod["codigos_nuevos"]:
            cod_id = nuevo["codigo"]
            if cod_id not in codigos_unicos:
                codigos_unicos[cod_id] = nuevo["descripcion"]
    
    print(f"\n📋 CÓDIGOS NUEVOS GENERADOS: {len(codigos_unicos)}")
    print(f"\n   {'ID':<6} DESCRIPCIÓN")
    print(f"   {'─'*6} {'─'*60}")
    for cod_id in sorted(codigos_unicos.keys()):
        print(f"   {cod_id:<6} {codigos_unicos[cod_id]}")
    
    print(f"\n   Total respuestas con códigos nuevos: {len(nuevos_con_codigos)}")

# 🆕 MEJORA: Mostrar métricas al final
if "metricas" in resultado_final_v3:
    imprimir_metricas(resultado_final_v3["metricas"])

🔍 VERIFICACIÓN PRE-EJECUCIÓN

📊 Respuestas cargadas: 14
📚 Catálogo histórico: 9 códigos

✅ Catálogo cargado correctamente
   Primeros 3 códigos:
   • [90] Ninguno
   • [91] No Recuerda
   • [92] No Sabe

🧮 Batches esperados: 2
⚙️  Límite de recursión: 100
🔢 Código inicial para nuevos códigos: 1
   (Códigos especiales 90-98 excluidos del cálculo)

🚀 EJECUTANDO GRAFO V3

📊 Respuestas: 14
📦 Batch size: 10
🤖 Modelo: gpt-5



✅ Validando 10 respuestas...
   🎯 Respuesta 8 detectada como código especial 90
   ✅ Válidas: 9/10

📊 Evaluando catálogo para 9 respuestas...
   ✅ Matches (confianza >= 0.95): 1

🔍 Analizando cobertura para 9 respuestas...
   (Códigos nuevos empezarán desde: 1)
   🔗 Agrupando 2 conceptos similares bajo código 4: Confianza en la marca
      Originales: Familiaridad con la marca, Confianza en la marca
   ✅ Duplicados detectados y agrupados: 1
   ✅ Completamente cubiertas: 2/9
   🆕 Con conceptos nuevos: 7/9
   🆕 Total conceptos nuevos: 13

📝 Generando justificaciones...
 

In [27]:
# VISUALIZACIÓN DEL GRAFO V3
print("="*70)
print("📊 DIAGRAMA DEL GRAFO V3")
print("="*70)
print("")
print("Flujo del proceso:")
print("")
print("  START")
print("    ↓")
print("  preparar_batch  ← [LOOP: toma siguiente grupo de respuestas]")
print("    ↓")
print("  validar  → Filtrar respuestas basura")
print("    ↓")
print("  evaluar_catalogo  → Evaluar TODOS los códigos (True/False + confianza)")
print("    ↓")
print("  identificar_conceptos  → Detectar gaps (qué NO está cubierto)")
print("    ↓")
print("  justificar  → Explicar decisiones")
print("    ↓")
print("  ensamblar  → Combinar resultados")
print("    ↓")
print("  finalizar  → Incrementar batch_actual")
print("    ↓")
print("  ¿Hay más batches?")
print("    ├─ SÍ → volver a preparar_batch")
print("    └─ NO → END")
print("")
print("="*70)
print("")
print("🎯 VENTAJA CLAVE DE V3:")
print("")
print("  • evaluar_catalogo: Evalúa CADA código histórico explícitamente")
print("  • identificar_conceptos: Solo genera códigos para conceptos NO cubiertos")
print("  • Captura casos MIXTOS: Respuestas que necesitan históricos + nuevos")
print("")
print("="*70)

# Diagrama ASCII del grafo
print("\n📋 Diagrama ASCII del grafo:\n")
try:
    print(app_v3.get_graph().draw_ascii())
except ImportError:
    print("⚠️  Para ver el diagrama ASCII, instala: pip install grandalf")
    print("\n📊 Diagrama simplificado:\n")
    print("  ┌─────────────────┐")
    print("  │ preparar_batch  │")
    print("  └────────┬────────┘")
    print("           │")
    print("  ┌────────▼────────┐")
    print("  │    validar      │")
    print("  └────────┬────────┘")
    print("           │")
    print("  ┌────────▼────────────┐")
    print("  │  evaluar_catalogo   │")
    print("  └────────┬────────────┘")
    print("           │")
    print("  ┌────────▼──────────────────┐")
    print("  │ identificar_conceptos     │")
    print("  └────────┬──────────────────┘")
    print("           │")
    print("  ┌────────▼────────┐")
    print("  │   justificar    │")
    print("  └────────┬────────┘")
    print("           │")
    print("  ┌────────▼────────┐")
    print("  │    ensamblar    │")
    print("  └────────┬────────┘")
    print("           │")
    print("  ┌────────▼────────┐")
    print("  │   finalizar     │")
    print("  └────────┬────────┘")
    print("           │")
    print("     [loop o END]")

📊 DIAGRAMA DEL GRAFO V3

Flujo del proceso:

  START
    ↓
  preparar_batch  ← [LOOP: toma siguiente grupo de respuestas]
    ↓
  validar  → Filtrar respuestas basura
    ↓
  evaluar_catalogo  → Evaluar TODOS los códigos (True/False + confianza)
    ↓
  identificar_conceptos  → Detectar gaps (qué NO está cubierto)
    ↓
  justificar  → Explicar decisiones
    ↓
  ensamblar  → Combinar resultados
    ↓
  finalizar  → Incrementar batch_actual
    ↓
  ¿Hay más batches?
    ├─ SÍ → volver a preparar_batch
    └─ NO → END


🎯 VENTAJA CLAVE DE V3:

  • evaluar_catalogo: Evalúa CADA código histórico explícitamente
  • identificar_conceptos: Solo genera códigos para conceptos NO cubiertos
  • Captura casos MIXTOS: Respuestas que necesitan históricos + nuevos


📋 Diagrama ASCII del grafo:

                     +-----------+          
                     | __start__ |          
                     +-----------+          
                            *               
                            

In [ ]:
# EXPORTAR RESULTADOS

# Obtener nombre de la pregunta (columna de respuestas)
nombre_pregunta = resultado_final_v3["pregunta"]

# Crear diccionario para mapear fila_excel a ID
# Necesitamos cargar el archivo original para obtener los IDs
df_original = pd.read_excel(ARCHIVO_RESPUESTAS)
columna_id = df_original.columns[0]

# Mapeo: fila_excel -> ID
mapeo_id = {}
for idx, row in df_original.iterrows():
    fila_excel = idx + 2
    id_valor = row[columna_id]
    if pd.isna(id_valor):
        id_valor = idx + 1
    mapeo_id[fila_excel] = id_valor

# Construir datos para Excel 1: ID, [nombre_pregunta], Códigos asignados
datos_exportar = []

for cod in resultado_final_v3["codificaciones"]:
    # Obtener ID del mapeo
    fila_excel = cod["fila_excel"]
    id_valor = mapeo_id.get(fila_excel, fila_excel - 1)  # Fallback si no encuentra
    
    # Construir códigos asignados
    codigos_asignados = []
    
    # Códigos históricos (solo números)
    if cod["codigos_historicos"]:
        codigos_asignados.extend([str(c) for c in cod["codigos_historicos"]])
    
    # Códigos nuevos (solo números)
    if cod["codigos_nuevos"]:
        codigos_asignados.extend([str(nuevo["codigo"]) for nuevo in cod["codigos_nuevos"]])
    
    # Unir con punto y coma
    codigos_final = "; ".join(codigos_asignados) if codigos_asignados else ""
    
    # Construir fila de datos
    datos_exportar.append({
        "ID": id_valor,
        nombre_pregunta: cod["texto"],  # Columna con el nombre de la pregunta
        "Códigos asignados": codigos_final
    })

df_resultados = pd.DataFrame(datos_exportar)

# Excel 2: Crear catálogo de códigos nuevos (solo si hay códigos nuevos)
codigos_nuevos_unicos = {}
for cod in resultado_final_v3["codificaciones"]:
    if cod["codigos_nuevos"]:
        for nuevo in cod["codigos_nuevos"]:
            cod_id = nuevo["codigo"]
            if cod_id not in codigos_nuevos_unicos:
                codigos_nuevos_unicos[cod_id] = nuevo["descripcion"]

df_catalogo_nuevos = pd.DataFrame([
    {"COD": cod_id, "TEXTO": descripcion}
    for cod_id, descripcion in sorted(codigos_nuevos_unicos.items())
])

# Guardar en Excel con 2 hojas
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archivo_salida = project_root / "notebooks" / "resultados" / f"resultados_v3_{timestamp}.xlsx"
archivo_salida.parent.mkdir(exist_ok=True)

with pd.ExcelWriter(archivo_salida, engine='openpyxl') as writer:
    # Excel 1: Resultados
    df_resultados.to_excel(writer, sheet_name='Resultados', index=False)
    
    # Excel 2: Códigos Nuevos (solo si hay códigos nuevos)
    if not df_catalogo_nuevos.empty:
        df_catalogo_nuevos.to_excel(writer, sheet_name='Códigos Nuevos', index=False)

print("="*60)
print("📥 RESULTADOS EXPORTADOS")
print("="*60)
print(f"\n✅ Archivo: {archivo_salida.name}")
print(f"📊 Hojas:")
print(f"   • Resultados: {len(df_resultados)} filas")
print(f"     Columnas: ID, '{nombre_pregunta}', Códigos asignados")
if not df_catalogo_nuevos.empty:
    print(f"   • Códigos Nuevos: {len(df_catalogo_nuevos)} códigos")
    print(f"     Columnas: COD, TEXTO")
else:
    print(f"   • Códigos Nuevos: (vacío - no se generaron códigos nuevos)")

print(f"\n📋 Vista previa de Resultados (primeras 3 filas):\n")
print(df_resultados.head(3).to_string())
if not df_catalogo_nuevos.empty:
    print(f"\n📋 Vista previa de Códigos Nuevos:\n")
    print(df_catalogo_nuevos.head(10).to_string())

📥 RESULTADOS EXPORTADOS

✅ Archivo: resultados_v3_20251201_104548.xlsx
📊 Hojas:
   • Resultados: 10 filas
     Columnas: ID, 'P2', Códigos asignados
   • Códigos Nuevos: 13 códigos
     Columnas: COD, TEXTO

📋 Vista previa de Resultados (primeras 3 filas):

       ID                                                                                      P2 Códigos asignados
0  NUMERO  2.¿Por qué seleccionó la cara cuando vio esta imagen?Seleccionó cara cuando vio imagen                  
1      11                          por que me sorprendió que tuvieran artos productos y elegantes           1; 2; 3
2      13                                     porque ya conocio la marca, asi que no me sorprendi                 4

📋 Vista previa de Códigos Nuevos:

   COD                   TEXTO
0    1                Sorpresa
1    2   Variedad de productos
2    3   Presentación elegante
3    4   Confianza en la marca
4    5  Agrado por el producto
5    6   Confianza en la marca
6    7                 Em

: 

## 📋 División de Prompts por Nodo

### **Nodo 1: VALIDAR** ✅
- **Responsabilidad**: Filtrar respuestas basura
- **Prompt**: Simple, solo reglas de rechazo/aceptación
- **No necesita** reglas de especificidad

### **Nodo 2: EVALUAR_CATALOGO** ✅
- **Responsabilidad**: Evaluar si códigos históricos aplican (booleano)
- **Prompt**: Mejorado con reglas de evaluación:
  - Considera nivel de especificidad (busca idea central, no palabras exactas)
  - Permite múltiples códigos por respuesta
  - Guía de confianza (0.7-1.0 para aplicar)
  - Enfoque conservador (mejor dejar sin código que asignar incorrecto)

### **Nodo 3: IDENTIFICAR_CONCEPTOS** ✅
- **Responsabilidad**: Crear códigos nuevos o identificar gaps
- **Prompt**: **COMPLETO** con todas las reglas de especificidad de `gpt_hibrido.py`
- **Incluye**: Nivel de especificidad, agrupación, unicidad, etc.

### **Nodo 4: JUSTIFICAR** ✅
- **Responsabilidad**: Generar justificaciones breves
- **Prompt**: Muy simple, solo pide ser conciso

### **Nodo 5: ENSAMBLAR** ✅
- **Responsabilidad**: Combinar resultados
- **Sin prompt**: Solo lógica de ensamblaje

**Conclusión**: Los prompts están divididos correctamente. El Nodo 3 tiene todas las reglas críticas para crear códigos nuevos.

---

## 🎯 Resumen de Mejoras V3

### ✅ Códigos Secuenciales Globales
- Los códigos nuevos son **secuenciales entre batches**
- Contador global `proximo_codigo_nuevo` persiste durante toda la ejecución
- Si hay catálogo histórico → empieza desde (max_codigo + 1)
- Si NO hay catálogo → empieza desde 1

### 🤖 GPT-5 Disponible
- Modelo por defecto: **gpt-5** (disponible desde 2025)
- Otros modelos: gpt-4o, gpt-4o-mini, gpt-4-turbo

### 📊 Exportación Mejorada
- **2 hojas en Excel:**
  - Resultados: Respuestas con códigos asignados
  - Códigos Nuevos: Catálogo con IDs numéricos (COD, TEXTO)

### 🔢 Ejemplo de Secuencialidad:
```
Batch 1: Genera códigos 1, 2, 3
Batch 2: Genera códigos 4, 5, 6  ✅ (no reinicia)
Batch 3: Genera códigos 7, 8     ✅ (continúa)
```
